In [1]:
!nvidia-smi

Mon Jul 19 07:48:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d tapakah68/supervisely-filtered-segmentation-person-dataset

!unzip -q /content/supervisely-filtered-segmentation-person-dataset.zip

Saving kaggle.json to kaggle.json
kaggle.json
100% 4.30G/4.31G [01:24<00:00, 25.6MB/s]
100% 4.31G/4.31G [01:24<00:00, 54.7MB/s]


In [3]:

!pip install q tensorflow==2.1
!pip install q keras==2.3.1
!pip install git+https://github.com/qubvel/segmentation_models

# for str decode error ... run it and restart runtime
!pip uninstall h5py
!pip install h5py==2.10.0

     |████████████████████████████████| 421.8MB 16kB/s 
     |████████████████████████████████| 3.9MB 37.7MB/s 
     |████████████████████████████████| 450kB 53.5MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=57f999de125516d13167b908d0ceed1d760872063f0a3734061c8dcb981ad332
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.13.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tens

In [9]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf

def load_dataset(dataset_path):
    images = glob(os.path.join(dataset_path, "images/*"))
    masks = glob(os.path.join(dataset_path, "masks/*"))

    train_x, test_x = train_test_split(images, test_size=0.2, random_state=1996)
    train_y, test_y = train_test_split(masks, test_size=0.2, random_state=1996)

    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = load_dataset('/content/supervisely_person_clean_2667_img')
print(len(train_x), len(train_y), len(test_x), len(test_y))

train_X_y_paths = list(zip(train_x, train_y))
val_X_y_paths = list(zip(test_x, test_y))

def shuffle(samples):
  index = np.random.permutation(len(samples))
  return np.array(samples)[index]

# def get_data(base_dir,w):
#   img = glob(base_dir+f'images/{w}/*/*.png')
#   img_names = [path.split(f'/{w}/')[1].split('_leftImg8bit.png')[0] for path in img]
#   label = [base_dir + f'gtFine/{w}/' + name + '_gtFine_color.png' for name in img_names]
#   del img_names
#   return img, label

# base_dir = "/content/people_segmentation"
# train_img,train_label = pre_shuffle(*get_data(base_dir,"images"))
# val_img,val_label = pre_shuffle(*get_data(base_dir,"val"))

def get_data_generator(samples,batch_size):

  while True:
    for offset in range(0, len(samples), batch_size): 
      samples = shuffle(samples)           
      batch_samples = samples[offset:offset+batch_size]
      X_train = []
      Y_train = []        
 
      for batch_sample in batch_samples:
          
        X_image = cv2.imread(batch_sample[0])[:,:,::-1]# convert bgr to rgb
        X_image = cv2.resize(X_image, (512, 512))
        X_image = X_image / 255.0
        X_image = X_image.astype(np.float32)

        Y_image = cv2.imread(batch_sample[1], 0)
#         Y_image = Y_image * 255.0
        Y_image = cv2.resize(Y_image, (512, 512))
        Y_image = Y_image / 255.0
        Y_image = Y_image.astype(np.float32)
        Y_image = np.expand_dims(Y_image, axis=-1)

        X_train.append(X_image)
        Y_train.append(Y_image)
      yield np.array(X_train), np.array(Y_train)


BATCH_SIZE = 16
train_generator = get_data_generator(train_X_y_paths,batch_size=BATCH_SIZE)
val_generator = get_data_generator(val_X_y_paths,batch_size=BATCH_SIZE)
 
x_train, y_train = next(train_generator)
x_val, y_val  = next(val_generator)
 
print(x_train.shape,y_train.shape)
print(x_val.shape, y_val.shape)

2133 2133 534 534
(16, 512, 512, 3) (16, 512, 512, 1)
(16, 512, 512, 3) (16, 512, 512, 1)


In [10]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
%env SM_FRAMEWORK=tf.keras

import numpy as np
import tensorflow as tf
# from model import build_unet
# from data import load_dataset, tf_dataset
from segmentation_models.losses import bce_jaccard_loss, dice_loss, JaccardLoss
from segmentation_models.metrics import iou_score, f1_score, precision, recall
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from DeepLabV3p import Deeplabv3

""" Hyperparamaters """
dataset_path = "people_segmentation"
input_shape = (512, 512, 3)
batch_size = 8
epochs = 100
lr = 1e-2
# model_path = "unet.h5"
# csv_path = "data.csv"

""" Load the dataset """
# (train_x, train_y), (test_x, test_y) = load_dataset(dataset_path)
# print(f"Train: {len(train_x)} - {len(train_y)}")
# print(f"Test: {len(test_x)} - {len(test_y)}")

# train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
# test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

""" callbacks """
checkpoint_filepath = '/content/Deeplabv3p_MobilenetV2_Human_Seg_000+{epoch}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_iou_score',
    mode='max',
    verbose = 1,
    period = 5,
    save_best_only=False
    )

callbacks = [
    model_checkpoint_callback,
    ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
#     CSVLogger(csv_path),
    EarlyStopping(monitor="val_loss", patience=10)
]

""" steps per epochs """
train_steps = len(train_x)//batch_size
if len(train_x) % batch_size != 0:
    train_steps += 1

test_steps = len(test_x)//batch_size
if len(test_x) % batch_size != 0:
    test_steps += 1

print(train_steps, test_steps)


""" Model training """
ls = dice_loss + bce_jaccard_loss
metrics = [precision, recall, f1_score, iou_score] 

model = Deeplabv3(weights='cityscapes', input_tensor=None, input_shape=(512,512,3), classes=1, 
                  backbone= 'mobilenetv2', #'xception'
                  OS=16, alpha=1., activation='sigmoid')

for layer in model.layers:
  # if layer.name == "decoder_conv0_depthwise":
  #   break
  # else:
  layer.trainable = True
# for layer in model.layers:
#   print(layer.name,layer.trainable)

model.compile(
    loss=ls,
    optimizer=tf.keras.optimizers.Adam(lr),
    metrics=metrics
)
# model.summary()

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks
)


env: SM_FRAMEWORK=tf.keras
267 67
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 267 steps, validate for 67 steps
Epoch 1/100
267/267 [==============================] - 529s 2s/step - loss: 1.4936 - precision: 0.5663 - recall: 0.6662 - f1-score: 0.6069 - iou_score: 0.4404 - val_loss: 4.0646 - val_precision: 0.4458 - val_recall: 0.0037 - val_f1-score: 0.0072 - val_iou_score: 0.0036
Epoch 2/100
267/267 [==============================] - 520s 2s/step - loss: 1.1611 - precision: 0.6799 - recall: 0.7476 - f1-score: 0.7078 - iou_score: 0.5495 - val_loss: 4.4901 - val_precision: 0.3791 - val_recall: 0.9669 - val_f1-score: 0.5422 - val_iou_score: 0.3739
Epoch 3/100
267/267 [==============================] - 513s 2s/step - loss: 1.0136 - precision: 0.7304 - recall: 0.7833 - f1-score: 0.7511 - iou_score: 0.6032 - val_loss: 2.0297 - val_precision: 0.4137 - val_recall: 0.4340 - val_f1-score: 0.4206 - val_iou_score: 0.2671
Epoch 4/100
267/267 [==============================] - 521s 2s/

# Predictions

In [71]:
#@title
model = Deeplabv3(weights='cityscapes', input_tensor=None, input_shape=(512,512,3), classes=1, 
                  backbone= 'mobilenetv2', #'xception'
                  OS=16, alpha=1., activation='sigmoid')
# from tensorflow.keras.models import load_model
model.load_weights('/content/Deeplabv3p_MobilenetV2_Human_Seg_000+30.h5')
model.compile(
    loss=ls,
    # optimizer=tf.keras.optimizers.Adam(lr),
    metrics=metrics
)

In [72]:
#@title
img = cv2.imread('SJ.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [73]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [74]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_SJ.jpg',pred)

True

In [75]:
#@title
img = cv2.imread('SJ.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_SJ.jpg',overlay)

True

In [76]:
#@title


In [77]:
#@title
img = cv2.imread('75805310.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [78]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [79]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_75805310.jpg',pred)

True

In [80]:
#@title
img = cv2.imread('75805310.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_75805310.jpg',overlay)

True

In [81]:
#@title


In [82]:
#@title
img = cv2.imread('oldestman.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [83]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [84]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_oldestman.jpg',pred)

True

In [85]:
#@title
img = cv2.imread('oldestman.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_oldestman.jpg',overlay)

True

In [86]:
#@title


In [87]:
#@title
img = cv2.imread('multiple.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [88]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [89]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_multiple.jpg',pred)

True

In [90]:
#@title
img = cv2.imread('multiple.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_multiple.jpg',overlay)

True

In [91]:
#@title


In [92]:
#@title
img = cv2.imread('2.jpg')
img = cv2.resize(img, (512, 512))
img = img/255.0
img = img.astype(np.float32)
img = img.reshape(1, 512, 512, 3)
img.shape

(1, 512, 512, 3)

In [93]:
#@title

pred = model.predict(img)
pred = pred.reshape(512, 512)
pred.shape

(512, 512)

In [94]:
#@title
pred = pred * 255
pred = pred.astype(np.uint8)
cv2.imwrite('pred_2.jpg',pred)

True

In [95]:
#@title
img = cv2.imread('2.jpg')
img = cv2.resize(img, (512, 512))

pred = np.dstack([pred, pred, pred])
overlay = cv2.addWeighted(img, 0.6, pred, 0.4, 0)
cv2.imwrite('overlay_2.jpg',overlay)

True